In [1]:
import os
import datetime
from datetime import timedelta
import csv
import time
import re
import string
import requests
import numpy as np
from urllib.parse import urlencode
import selenium
from selenium.webdriver import Firefox
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by  import   By 
from selenium.webdriver.firefox.options  import   Options 
from selenium.webdriver.support import   expected_conditions  as   EC 
from selenium.webdriver.support.wait  import   WebDriverWait 

In [2]:
"""cities=["L\'Aquila"
        ,'Teramo','Pescara','Chieti','Potenza','Matera','Cosenza','Catanzaro','Cosenza','Crotone','Vibo Valentia',
        'Reggio Calabria','Caserta','Benevento','Avellino','Salerno','Napoli','Piacenza','Parma','Reggio Emilia','Modena',
        'Ferrara','Ravenna','Rimini','Forli-Cesena Area','Bologna','Udine','Gorizia','Trieste','Pordenone','Viterbo','Rieti',
        'Latina','Frosinone','Roma','Imperia','Savona','La Spezia','Genova','Varese','Como','Sondrio','Bergamo','Brescia','Pavia',
        'Cremona','Mantova','Lecco','Lodi','Milano','Pesaro e Urbino','Ancona','Macerata','Ascoli Piceno',
        'Fermo','Campobasso','Isernia','Vercelli','Novara','Cuneo','Asti','Alessandria','Biella','Verbano-Cusio-Ossola','Torino',
        'Foggia','Taranto','Brindisi','Lecce','Barletta-Andria-Trani','Bari','Sassari','Nuoro','Oristano','Sardegna del Sud','Cagliari',
        'Trapani','Agrigento','Caltanissetta','Enna','Ragusa','Siracusa','Palermo','Messina','Catania','Lucca',
        'Pistoia','Livorno','Pisa','Arezzo','Siena','Grosseto','Prato','Firenze', 'Massa Carrara Area',
        'Bolzano','Trento','Perugia','Terni','Aosta','Verona','Vicenza','Belluno','Treviso','Padova',
        'Rovigo','Venezia'
       ]
       """
cities=["Lucca"]

In [12]:
date = datetime.date.today()
dataOdierna=date.strftime("%d/%m/%Y")
timeShort=3 #variabile attesa esplicita caricamento pagina

def preparaDriver(link):
    options = Options()
    options.headless = True  #modalita senza interfaccia grafica
    options.add_argument('--lang=it')
    driver = Firefox(options=options) #invoco il webdriver di firefox geckodriver in path
    driver.get(link)
    wait = WebDriverWait(driver, timeout=10).until( #attesa esplicita
        EC.presence_of_all_elements_located(
            (By.CLASS_NAME, 'sr-hotel__title')))
    return driver

def scrape_risultati(driver): 
    clickCookie=driver.find_element_by_xpath('//*[@id="cookie_warning"]/div/div/div[2]/button/span').click()
    time.sleep(timeShort) #richiede qualche secondo di attesa
    clickTipologiaHotel=driver.find_element_by_xpath('//span[@class="filter_label"][contains(text(),"Hotel")]').click()
    #con questo evento viene selezionata solo la tipologia di struttura hotel
    time.sleep(timeShort)
    
    alloggiCitta = list() #lista dove salvare i risultati
    indiceAlbergo=0
    for accomodation in driver.find_elements_by_class_name('sr_item_content'):
        Hotel=dict() #dizionario per appendere le informazioni dei singoli hotel ai relativi prezzi
        htmlText=accomodation.get_attribute('innerHTML')
        print("indice albergo: " + str(indiceAlbergo))
        prezzoStrutture=(re.findall('(€&nbsp;\d*)+', htmlText)[0])[indiceAlbergo]
        Hotel['nome Hotel'] = driver.find_element_by_xpath('//*[@id="hotellist_inner"]/div[1]/div[2]/div[1]/div[1]/div[1]/h3/a/span[1]').text #restituisce il nome hotel
        Hotel['data odierna']=dataOdierna
        Hotel['citta']=stringaCitta  #nome della città
        Hotel['disponibilita odierna']=numstruttureTrovate
        Hotel['prezzo']=prezzoStrutture
        alloggiCitta.append(Hotel)
        indiceAlbergo+=1
    return alloggiCitta   
    print (prezzoStrutturePul)

if __name__ == '__main__':

    try:
        for i in range(7,35,7): #funzione che permette di scandire per intervalli start, stop, step
            print("Giorni: " + str(i))
            dataCheckin=date + timedelta(days=i) #da  verificare se prende il settimo giorno
            dataCheckout=dataCheckin + timedelta(days=1)
            for city in cities:    
                parametri = { 
                    'checkin_month': dataCheckin.month,
                    'checkin_monthday':dataCheckin.day,
                    'checkin_year':dataCheckin.year,
                    'checkout_month':dataCheckout.month,
                    'checkout_monthday':dataCheckout.day,
                    'checkout_year':dataCheckout.year,
                    'group_adults':2,
                    'group_children':0,
                    'dest_type': 'city',
                    'ss': city,
                    'order' : 'distance_from_search'
                    }
            queryString = urlencode(parametri) #libreria urlLib per formattare la url
            url="https://www.booking.com/searchresults.html?%s" % queryString
            driver = preparaDriver(url)
            stringaCitta=city #variabile da assegnare alla lista degli alloggiCitta
            xpathStruttura = driver.find_element_by_xpath('//*[@role="heading"]').text  #percorso del titolo che varia a seconda delle citta
            numstruttureTrovate=(re.findall('[\d]+',xpathStruttura)[0]) #prende il primo numero in lista relativo alle strutture
            alloggiCitta = scrape_risultati(driver) #quanti hotel prendo in considerazione per ogni città
            keys = alloggiCitta[0].keys()
            with open('scrapingBooking.csv', 'a', newline='') as csvFile: #appende ai dati precedenti 
                dict_writer = csv.DictWriter(csvFile, 1) # imposto i nomi dei campi
                dict_writer.writeheader(keys)
                dict_writer.writerows(alloggiCitta)
                driver.close()
    finally:
        driver.quit()

7
indice albergo: 0
indice albergo: 1
indice albergo: 2
indice albergo: 3
indice albergo: 4
indice albergo: 5
indice albergo: 6
indice albergo: 7
indice albergo: 8
indice albergo: 9
indice albergo: 10


IndexError: string index out of range